<img src="./images/build_ai_job_application_helper_app.png" width="800" />

globalnerdy.com/job-app

# Welcome to _Build an AI job application helper app_!

This Jupyter Notebook is from [Tampa Bay Artificial Intelligence Meetup’s](https://www.meetup.com/tampa-artificial-intelligence-meetup/) exercise from [our session on Monday, March 17, 2025](https://www.meetup.com/tampa-artificial-intelligence-meetup/events/306353685/?eventOrigin=group_upcoming_events), which took place at [Embarc Collective](https://embarccollective.com/) in [Tampa](https://en.wikipedia.org/wiki/Tampa,_Florida). It was organized by [Joey de Villa](https://www.linkedin.com/in/joeydevilla/) and [Anitra Pavka](https://www.linkedin.com/in/anitrapavka/).

In that session, we demonstrated how to build an application to help people with the job search process. Feel free to use it in your job search, and feel free to modify it to suit your job search needs!

## What is this “Jupyter Notebook” thing?

<img src="./images/jupyter_logo.png" width="250" />

A _Jupyter Notebook_ is a document that combines two kinds of information in one place:

- **Content:** Text, graphics, and any other media that you can put into a web page.
- **Code:** Code that you can not only run, but edit too!

I like to think of Jupyter Notebooks as both:

- Code that comes with a lot of documentation built in
- Literature that you can also execute

I often use Jupyter Notebooks in presentations, treating them as slide decks that also contain the code for my demos. I can give copies away, so that people who see my presentations can see my slides and run my code long after the presentation’s over.

This very document you’re reading is a Jupyter Notebook. It’s made up of cells, and you should think of it as being like a column in a spreadsheet — a vertical list of cells:

<img src="./images/jupyter_notebox_as_a_spreadsheet_column.png" width="400" />

Each cell in a Jupyter Notebook can contain one of two kinds of data:

- **Markdown:** Content, such as text, graphics, and other media that you can put into a web page, written in Markdown, a way of formatting text that’s simpler than HTML.
- **Code:** Usually Python, but Jupyter Notebook supports other programming languages.

Each cell can be _executed_ or _run_, in the sense of running a program, where:

- Running a **Markdown** cell renders the content of that cell, and
- Running a **Code** cell executes the content of that cell.

By using a Jupyter Notebook, I can give you an application that has all sorts of notes embedded in it, or if you prefer, notes that have an application embedded in them. Either way, I hope makes it easier for you to use and customize to suit your own needs.

## The job-seeker’s challenge

<img src="./images/you_know_what_this_is_like.png" width="600" />

According to the Silicon Valley-based career guidance service Pathrise, [job seekers who sent 20+ job applications every week got more interviews and landed a job sooner](https://www.pathrise.com/guides/how-long-does-it-take-to-find-a-job/).

That’s a lot of work, especially since the general advice is to [customize your resume for every job application](https://resume.io/blog/customize-resume-for-each-application). Wouldn’t it be nice if there were a way to get some help customizing your resume for every job application you have to fill out?

This application sets out to solve that problem. It uses an AI — more specifically, a large language model (LLM) — to perform the repetitive task of customizing your resume for a specific job description.

### How it works

The _incredibly scientific_ diagram below illustrates how the application will work:

<img src="./images/how_it_works.png" width="1200" alt="Playful diagram with arrows leading a job description and resume to Jupyter Notebook and Python, and from there, and arrow leading to a number of LLMs, and from there an arrow leading to a spakling custom resume with a cartoon unicon 'dabbing' in front of it." />

This application takes two inputs:

1. **The job description for the job you’re applying for.** You’ll copy this from LinkedIn or whatever site had the job description and paste it into the application.
2. **Your resume, in Markdown format.** Putting it in Markdown format is the simplest way to convey formatting such as headings, bold and italics, links, and so on to the LLM.

The application will use these inputs to construct a prompt, which it will send to the LLM. It will receive the LLM’s response, which it will then save as a file to your computer

## Setup

### An OpenAI account
This notebook 
Because getting everyone in this meetup to set up and OpenAI API account and an API key would probably take all the time allotted for the meetup, we’re going to keep things simple: **in this meetup, you’ll use Joey’s OpenAI account and an API key that will be active only during the meetup.** In order to run this application after the meetup, you’ll need the following:

- **An OpenAI account.** You can sign up for one at the [OpenAI Platform page](https://platform.openai.com/).
- **A secret key.** Log into OpenAI and visit the [API keys page](https://platform.openai.com/api-keys), and create a new secret key by clicking the **Create New Secret Key** button.
- **Some money in your OpenAI credit balance.** As it says on the [Billing page](https://platform.openai.com/settings/organization/billing/overview), “When your credit balance reaches $0, your API requests will stop working.” Fortunately, one or two US dollars should be enough to cover running this application many, many times.

Once you have created a secret key, copy that key. Open the `.env` file located in the same directory as this notebook and look for this line:

```
OPENAI_API_KEY = replace_this_with_your_secret_key
```

> Filenames that begin with `.` (period) characters are system files, and for your safety, macOS hides them. You can make them visible by using the ⌘-shift-. keyboard shortcut (that is, hold down command, shift, and the `.` keys at the same time) when looking at a Finder window or when opening files.

Replace `replace_this_with_your_secret_key` with your secret key and save the file to the directory where you will create the Jupyter notebook for this demo. By storing the API key in a .env file (and making sure that the .env is _not_ added to source control), you avoid “hard-coding” the API key into your application, which makes it more likely that someone unauthorized will discover it.

The process is similar for other LLMs, such as Claude, Deepseek, and Writer.


### Install the Necessary Packages

There’s no need to reinvent the wheel when we Python has a large collection of packages to take care of things so we don’t have to code them ourselves. Let’s install the following packages:

- `dotenv`: Provides functions for reading `.env` files to create environment variables. We’ll use this to get the API keys so that the application can talk to LLMs.
- `markdown-pdf`: Converts Markdown documents into PDF documents. We’ll use this to generate the final copy of the customized résumé.
- `openai`: Provides functionality for communicating with OpenAI’s AI models.
- `pyperclip`: Gives our application access to the clipboard.

The command below installs these packages, which are listed in the file `requirements.txt`. If it doesn’t work, try replacing `pip` with `pip3`:

In [ ]:
! pip install -r requirements.txt

## Create an OpenAI client object

The first step in building an OpenAI-powered application is to create an instance of the OpenAI client class. We’ll call it `client`, and it will enable access to OpenAI’s various APIs, including the one we’ll use: GPT.

In [3]:
import os
from dotenv import load_dotenv
from markdown_pdf import MarkdownPdf
from markdown_pdf import Section
from openai import OpenAI

# When called without arguments, OpenAI will try to get
# the API key from the OPENAI_API_KEY environment variable.
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
client = OpenAI()

## Create a completion function

Once we’ve created an OpenAI client object, let’s use it to create a _completion_. In the context of an LLM, a completion is the text that the model generates in response to a given prompt. It's essentially the output that the AI produces to “complete” the input it has been given.

In order to create a completion, we’ll use the `OpenAI` class’ `chat.completions.create()` method and provide it with the following information:

- **Model:** For the purposes of the meetup, we’ll use the **GPT-4o mini** model, a smaller, faster model that costs less to run. After the meetup, and once you’re using your own OpenAI account, you can change this out for a more capable (but more expensive) model.
- **Temperature:** A parameter that controls the randomness or predictability of the model's outputs. It's named after the temperature parameter in statistical physics, where higher temperatures lead to more disorder. It’s essentially a "creativity dial" that lets developers or users control how much the LLM sticks to the most likely responses versus exploring more varied possibilities. In most LLMs, this is a value between 0 and 2, and the default value is 1.
- **Messages:** A list of messages that make up the exchange or “conversation” with the LLM. Each message is a dictionary with two keys:
  - `role`: Defines the “speaker” of the message. It's essentially a label that identifies the source and purpose of each message in the exchange. It can be one of these three values:
    - `"system"`: The initial instructions or context given to the model. Sets the overall behavior, personality, and constraints for the LLM. Establishes rules and guidelines for how the model should respond. Often includes information about the model's capabilities and limitations. Not directly visible to users in most interfaces.
    - `"user"`: Represents the person interacting with the LLM. Provides queries, instructions, or content for the model to respond to. Drives the direction of the conversation.
    - `"assistant"`: Represents the LLM's responses. Contains the generated completions from the model. Follows the guidelines established in the system role, and responds directly to user inputs.

In our test completion, let’s ask what the fastest bird in the world is.

In [4]:
def create_completion(
    system_prompt,
    user_prompt,
    model="gpt-4o-mini",
    temperature=1.0
):
    completion = client.chat.completions.create(
        model=model,
        temperature=temperature,
        messages=[
            {
                "role": "system",
                "content": system_prompt,
            },
            {
                "role": "user",
                "content": user_prompt,
            },
        ],
    )
    return completion

In [7]:
test_completion = create_completion(
    "You are a helpful assistant",
    "What’s the fastest fish in the world?",
)

Now that we have the completion, let’s see what it contains.

In [8]:
print(test_completion)

ChatCompletion(id='chatcmpl-BeKvUYa8juHOnpPwrYRDw6tgvLit4', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The fastest fish in the world is the black marlin (Istiompax indica). It is capable of swimming at speeds up to 82 miles per hour (132 kilometers per hour). Other fast fish include the sailfish and the swordfish, but the black marlin holds the title for the fastest.', role='assistant', function_call=None, tool_calls=None, refusal=None, annotations=[]))], created=1748953260, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_34a54ae93c', usage=CompletionUsage(completion_tokens=62, prompt_tokens=25, total_tokens=87, prompt_tokens_details={'cached_tokens': 0, 'audio_tokens': 0}, completion_tokens_details={'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}))


That’s a lot. Let’s extract the part we _really_ care about: the answer.

In [14]:
def create_completion(
    system_prompt,
    user_prompt,
    model="gpt-4o-mini",
    temperature=1.0
):
    completion = client.chat.completions.create(
        model=model,
        temperature=temperature,
        messages=[
            {
                "role": "system",
                "content": system_prompt,
            },
            {
                "role": "user",
                "content": user_prompt,
            },
        ],
    )
    return completion.choices[0].message.content

In [15]:
test_completion = create_completion(
    "You are a helpful assistant",
    "What’s the fastest bird in the world?",
)
print(test_completion)

The fastest bird in the world is the peregrine falcon (Falco peregrinus). When in a dive, known as a stoop, it can reach speeds of over 240 miles per hour (386 kilometers per hour). This incredible speed makes it not only the fastest bird but also the fastest animal on the planet when considering speed attained during a dive.


Now that we have a working client to communicate with the LLM, let’s build the rest of the program.

## Define the base résumé

The application needs a “base résumé.” It will compare the base résumé against the job description in order to generate a customized résumé.

The base résumé should be in Markdown format, which allows us to specify formatting such as headings, bold and italics, bullet points, and even hyperlinks. This formatting will be useful when converting the résumé into a PDF or Microsoft Word file.

The base résumé will be stored in a constant called `BASE_RESUME`.

At the meetup, we used Joey’s résumé. Replace it with your own!

In [16]:
BASE_RESUME = """
# Joey de Villa

[joey@joeydevilla.com](mailto:joey@joeydevilla.com) • 813.330.9053 • Tampa, Florida 

- **Certifications:** CSM, A-CSM, CSD, CSPO from Scrum Alliance 
- **Education:** B.Sc. Computer Science, Queen’s University, Kingston, Canada 
- **Primary programming languages:** Kotlin, Python, Swift
- **Technology blog:** globalnerdy.com 
- **LinkedIn:** linkedin.com/in/joeydevilla
- **GitHub:** github.com/AccordionGuy/


## Consultant @ Atypical Consulting, LLC
July 2020 - present • Tampa, FL 

- **Developer Relations at [Writer](https://writer.com/) (San Francisco), August 2024 - present:**
    - Wrote documentation for Writer’s AI platform, including:
        - Performing code reviews of developer toolkits, primarily writer-python
        - Developer documentation for their Palmyra LLM APIs
        - Developer cookbooks featuring code examples that demonstrate applications of their APIs
        - Tech editing articles on the Writer Engineering blog
- **Python/AI Instructor at [Computer Coach Training Center](https://www.computercoach.com/) (Tampa), July 2020 - present:**
    - Designed and taught these courses::
        - Intro to Python and AI Development (40-hour online course, July - August, 2024)
        - Intro to AI Application Development (6-hour online course, June 11 - 13, 2024)
        - Intro to Python (3 40-hour online courses, July 2020 /  September 2022 / November 2023)
        - Intro to JavaScript/React (40-hour online course, starting September 2020)
- **Developer Relations Consultant at [Unified.to](http://unified.to/) (Toronto, Canada), April - June 2024:**
    - Co-developed Unified.to’s go-to-market strategy for their Unified API for AI/LLM services.
    - Worked with the product owner on a strategy for developer outreach and started executing it.
    - Co-wrote their white paper, [_The State of SaaS APIs 2024_](https://unified.to/blog/the_state_of_saas_apis_2024).
    - Revised their documentation, which while complete, needed more explanatory text and examples.
    - Created articles for their blog and videos based on those articles for their YouTube channel. These articles and videos covered Unified.to’s unified API technology and features, and developing applications using external APIs.
    - Doubled Unified.to’s LinkedIn following in two months.
    - Wrote a proposal for a presentation that was accepted for Nordic APIs Platform Summit 2024 in Stockholm.
- **Independent Technical Presenter/Guest Speaker:**
    - [_Make Smarter AI Apps with RAG_](https://www.youtube.com/watch?v=F2dm0ufm7ZA) @ Civo Navigate Local Tampa 2024, April 16, 2024
    - [_xz Made EZ_](https://www.youtube.com/watch?v=tIbWFC1LauY) @ BSides Tampa, April 6, 2024
    - [_AI: How to Jump In Right Away_](https://www.youtube.com/watch?v=eIErTksBrdo) @ Austin Forum on Technology and Society, April 2, 2024 (online)
    - [_You’re Not Too Late to the AI Party_](https://www.youtube.com/watch?v=xjGuNY6bmCs) @ Civo Navigate North America 2024, February 20, 2024 (Austin, Texas)
    - [_How Computers Work “Under the Hood”_](https://www.youtube.com/watch?v=B9WyrVabbeo) @ Tampa Devs, November 15, 2023
- **Publisher/editor/author of the technology weblog Global Nerdy, August 2006 - present:**
    - Creator of the Tampa Bay Tech Events List, a weekly feature of Global Nerdy that presents the upcoming week’s technology, entrepreneur, and “nerd” events taking place in the Tampa Bay area. The list is generated using my Python script that scrapes Meetup and local event sites.


## Author / Technical Editor / Presenter • Kodeco (formerly RayWenderlich.com)
July 2017 - present • Tampa, FL 

- Author of the [_Python for AI: A Crash Course_](https://www.kodeco.com/ai/paths/python) lesson series.
- Co-author of the book [_iOS Apprentice, 8th edition_](https://www.kodeco.com/books/ios-apprentice/v8.3). Updated two major sections of the book from UIKit to SwiftUI and rewrote two of the four apps presented in the book to reflect the new material. 
- Article author:
    - [_Layoff to Liftoff: Surviving Downsizing in the Tech Industry](https://www.kodeco.com/44653678-layoff-to-liftoff-surviving-downsizing-in-the-tech-industry)
    - [_Jetpack Compose Tutorial for Android: Getting Started_](https://www.kodeco.com/40153513-jetpack-compose-tutorial-for-android-getting-started)
    - [_Beginning Data Science with Jupyter Notebook and Kotlin_](https://www.kodeco.com/27470499-beginning-data-science-with-jupyter-notebook-and-kotlin)
    - [_Create Your Own Kotlin Playground (and Get a Data Science Head Start) with Jupyter Notebook_](https://www.kodeco.com/27470305-create-your-own-kotlin-playground-and-get-a-data-science-head-start-with-jupyter-notebook)
    - [_Kotlin Cheat Sheet and Quick Reference_](https://www.kodeco.com/6362971-kotlin-cheat-sheet-and-quick-reference)
    - [_What’s New in Kotlin 1.3_](https://www.kodeco.com/3871338-what-s-new-in-kotlin-1-3)
    - [_Augmented Reality in Android with Google’s Face API_](https://www.kodeco.com/523-augmented-reality-in-android-with-google-s-face-api)
- Author and presenter of [_Beginning ARKit_](https://www.kodeco.com/737368-beginning-arkit), a video course covering augmented reality app development for iOS devices. Designed and developed 4 original apps for this course.
- Technical editor for ARKit by Tutorials.
- Presenter: Highest-rated speaker at the RWDevCon 2018 conference with 2 presentations on ARKit: a [two-hour introductory presentation](https://www.youtube.com/watch?v=0bea2V8Jrw4), and an [in-depth four-hour workshop](https://youtu.be/_sjyaDhKk2Q).


## Developer Advocate @ [Okta](https://www.okta.com/)
October 2020 - April 2024 • Working remotely from Tampa, FL

- Led the effort and worked with product owners to update Okta’s Auth0 mobile quickstart kits for .NET, iOS, and Android, including architecture, visual interface, and documentation.
- Created [articles](https://auth0.com/blog/authors/joey-devilla/) and videos covering Auth0, OAuth2, OIDC, security, and general programming for the Auth0 Developer Blog and the OktaDev YouTube channel, with a primary focus on mobile developers (iOS/Swift and Android/Kotlin), and a secondary focus on Python developers.
- Grew mobile article readership from nearly zero to over 20K viewers per month.
- Commissioned, reviewed, edited, and in some cases, rewrote the Auth0 Developer Blog’s first mobile development articles by guest authors, covering Flutter and React Native.
- Refreshed native mobile (Android and iOS) code samples that hadn’t been updated in 2 years, and brought them up to date by incorporating the newer native UI frameworks, SwiftUI and Jetpack Compose.
- Worked with the team behind the 2023 edition of Okta’s annual conference, Oktane, to create content for the conference’s mobile app, and wrote both [decision-maker-facing](https://www.okta.com/blog/2023/10/adding-passkeys-to-your-apps-with-okta-cic-powered-by-auth0/) and [developer-facing articles](https://auth0.com/blog/activate-passkeys-let-users-log-in-without-password/) for Auth0’s newly-announced support for passkeys. Demonstrated passkeys and other Auth0 features at the conference.
- Represented Okta at conferences, most notably PyCon US 2022 and 2023 and ng-conf 2022.


## Senior Mobile Developer @ [Lilypad](https://lilypad.co/)
September 2019 - April 2020 • Tampa, FL

- Cross-platform mobile developer for Lilypad, a sales/CRM app for the beverage alcohol industry.
- Worked with the product owner and back-end team to deliver major upgrades and bug fixes to the Android version, bringing it to feature, UI, and performance parity (60% speed improvement on some features) with the iOS version.
- Reduced Android version bug count by 50% in my first two months.
- Took charge of internal developer documentation. Devoted 20% of development time to writing much-needed internal documentation of the mobile app, refactoring, and upgrading the codebase to a more current version of JavaScript.
- Established and documented a coding standards/style guide.


## Lead Product Manager • [Sourcetoad](https://sourcetoad.com/)
July 2017 - June 2019 • Tampa, FL

- Worked with clients to architect custom web and mobile software for their businesses.
- Gathered requirements, and specified the functionality, workflows, user interfaces, and basic data schemas for the following customer software applications: 
    - Note-taking web application for pharmacists that reconciles patient’s medications and generates physician- and patient-facing recommendations.
    - App where customers submit prescriptions and participating pharmacies respond with offers and prices for the submitted prescriptions.
    - Investor-facing “Crunchbase for pharma” web application that tracks companies, conferences, and employees in the pharmacology and healthcare industries.
    - iPad-based self-serve kiosk for shipping via FedEx, UPS, or USPS.
    - “Reverse Angie’s List” mobile app for general contractors to rate their customers on their interactions and payments, and look up the ratings of prospective customers.
    - Mobile app for the construction industry that tracks tickets for the delivery and removal of construction material from sites. 
- Participated in sales meetings, showcasing the company’s application development capabilities and strengths to prospective clients, leading to sales, including the projects listed above. 
- Delivered technical talks at conferences and meetups, including BarCamp Tampa Bay, Tampa Hospitality Hackathon, and DevFest Florida. 
- Grew the company’s Twitter profile 10x, and wrote several articles for the company blog.
- Helped develop the company’s guidelines for interacting with clients.


## Technology Evangelist • [Smartrac](https://rfid.averydennison.com/en/home.html)
October 2016 - April 2017 • Columbia, MD — working remotely from Tampa

- Promoted Smartrac’s Smart Cosmos RFID platform to technical and non-technical audiences and provided technical information to developers who built solutions that integrated with the Smart Cosmos. 
- Wrote documentation and code examples for Smart Cosmos Objects, which connected real-world objects with RFID tags to their cloud-based data representations.
- Presented the Lifecycles solution at the booth at NRF’s “Retail’s Big Show 2017” conference. Brought in a major prospective customer ($100M revenue in 2016) for a meeting.
- Gathered requirements and wrote spec for a system for embedding RFID tags in luxury goods to track them through their lifetimes and prevent counterfeiting.


## Partner Technical Analyst / Evangelist • GSG (now Sakon)
March 2014 - September 2016 • Concord, MA — working remotely from Tampa

- Communications and technical specialist on IBM’s Network Infrastructure Cost Optimization (NICO) team, creating all their sales and marketing materials. Wrote, produced, and [narrated the official NICO promotional video](https://www.youtube.com/watch?v=2stdDHE0BNs).
- Product owner for the NICO Quick Assess web application for performing quick evaluations of an enterprise’s network infrastructure. 
- Created sales and marketing materials for GSG and its channel partners: presentations, case studies, white papers, sell sheets, videos, and even mobile applications. 
- Created documentation and training material for GSG’s SaaS applications, such as written documentation, demo scripts, and training videos. 
- Oversaw the website redesign, posted articles on the company blog, created promotional and informational videos on the company’s YouTube channel.
- Grew the company’s following on LinkedIn by 5x and Twitter by 10x.


## Chief Technology Officer • Comprehensive Technology Solutions
September 2012 - October 2013 • Toronto, Canada

- Ran the de facto managed mobility services department of Rogers Communications (the Canadian equivalent of Time-Warner), filling both CTO and marketing roles.
- Architected “BYOD in a Box” and “CL in a Box”, applications for small-/medium-sized businesses manage their BYOD and corporate-liable mobile devices.
- Performed mobile needs assessments for Rogers’ medium/large corporate customers.
- Helped customers with 1,000+ employees move them from corporate-liable to individual-liable devices. 
- Created Rogers-branded customer-facing marketing documents, including a white paper listing tips for businesses who want to implement a BYOD program, a guide to help enterprises create mobile device policies, and “sell sheets” for various Rogers services.


## Platform Evangelist • Shopify 
May 2011 - May 2012 • Ottawa and Toronto, Canada
Tools: Ruby on Rails

- Wrote the documentation and sample code (Ruby/JavaScript/CoffeeScript) for Shopify’s developer API, as well as documentation for store theme designers. Edited and wrote technical articles for Shopify's technology blog, whose primary audience was developers and designers.
- Managed a $1 million fund used to encourage developers to build apps for the Shopify platform.
- Represented Shopify at conferences, including BarCamp Tour 2011 in 9 U.S. cities.
- Managed the Twitter account and doubled the company’s followers.


## Developer Evangelist • Microsoft Canada 
October 2008 - April 2011 • Toronto, Canada
Tools: Visual Studio, ASP.NET MVC, SharePoint, Windows Mobile 6 / Windows Phone 7, XNA

- Wrote content and made presentations on Microsoft’s web and mobile technologies and evangelizing to “unfriendly” audiences. “Won over” developers and technology influencers who originally had strong negative opinions of Microsoft.
- Led the “breadth program” in the months leading up to Windows Phone 7’s release. Brought development tools, devices, documentation, and training to developers, coordinated the development of early apps by Canadian developers to pre-seed the Windows Phone app store, and organized Windows Phone developer events.
- Edited and wrote the lead articles for the Canadian edition of MSDN Flash, Microsoft's developer newsletter emailed every two weeks to 50,000 subscribers.
- Microsoft Canada’s most prolific blogger, writing almost 750 articles for Microsoft’s blog, Canadian Developer Connection. 
- Traveled across Canada, giving presentations and tutorials to developer, technologist, and student audiences on developing applications using Visual Studio, C# and Visual Basic, Windows 7, ASP.NET MVC, Windows Mobile, Windows Phone, and Xbox 360 as well as development techniques (test-driven development, introductory agile — XP, Scrum, and Kanban, and version control strategies). 
- Co-organized TechDays, a conference series in 8 Canadian cities. Organized a conference session in 2009, led one of the developer tracks in 2010, and ran all developer tracks in 2011. 
"""

## Provide a Job Description

The application also needs a job description. It will compare the base résumé against the job description in order to generate a customized résumé.

The base résumé will be stored in a constant called `BASE_RESUME`.

At the meetup, we used an actual job description from LinkedIn. Feel free to copy and paste over it.

In [ ]:
JOB_DESCRIPTION = """
RunPod is pioneering the future of AI and machine learning, offering cutting-edge cloud infrastructure for full-stack AI applications. Founded in 2022, we are a rapidly growing, well-funded company with a remote-first organization spread globally. Our mission is to empower innovators and enterprises to unlock AI's true potential, driving technology and transforming industries. Join us as we shape the future of AI.

The Developer Relations Team sits within the Product Team and ensures that we maintain a close connection between the RunPod community and our product. This team is dedicated to optimizing every aspect of the external developer's interaction with our products, aligning closely with RunPod's vision to be the compute backbone for AI/ML workloads at scale.

Developer Experience Engineers focus on identifying developer needs or pain points and building solutions to make their RunPod experience great. You'll create tools, sample projects, templates, demos, and workflows to enhance the developer experience across our platform. Additionally, you'll maintain our open source repositories and ensure that issues and pull requests are responded to in a timely manner. In this role, you'll help shape RunPod's developer experience and ensure that developers have the resources they need to succeed.

Responsibilities:

- Identify developer pain points, workflow optimizations, and onboarding improvements and work with the Product team to design solutions.
- Design and implement tooling, workflows, and sample projects to improve RunPod's developer experience.
- Participate in design and architectural discussions with the Product team to ensure that developers are represented in new features and services.
- Maintain open source repositories and documentation for tools, RunPod templates, and code samples.
- Respond to issues and pull requests on open source repositories in a timely manner.

Requirements:

- To be successful in this role, you'll need to understand and take ownership of developers' pain points as if they were your own.
- 6+ years of software development experience in an internal tooling, developer tooling, or developer experience role.
- Strong understanding of a major programming language and its ecosystem (Python, Go, or Rust preferred)
- Experience creating images and deploying with Docker
- Experience working with Product teams to triage customer issues and create requirements for projects

Preferred:

- Experience with AI and ML tooling and frameworks
- Active in one or more AI and ML communities
- Experience with RunPod
"""

## Create a system prompt

Next, we’ll define a system prompt, which is essentially a set of instructions that guides how the AI responds to people. Think of it as setting up the ground rules and personality for the AI before it starts a conversation.

Think of a system prompt like this:

1. **It's like a job description and behavioral guideline:** The system prompt tells the AI who it is, how it should behave, what tone to use, what it can and cannot discuss, and how to handle different types of questions.
2. **It provides background knowledge:** The system prompt may include specific facts that the AI should know about itself or various topics.
3. **It establishes boundaries:** The prompt defines what kinds of requests the AI should decline and how to politely redirect conversations when needed.
4. **It shapes the AI's "personality":** Whether the AI comes across as formal, casual, detailed, concise, humorous, or serious is largely influenced by the system prompt.

When you interact with an AI assistant, you're seeing the result of these behind-the-scenes instructions working together with the AI's training. The system prompt is invisible to users but shapes every response you receive.

The system prompt will be stored in a constant called `SYSTEM_PROMPT`.

In [ ]:
SYSTEM_PROMPT = """
You are an expert resume optimizer and cover letter writer designed to help job seekers present themselves
in the best possible light for specific job opportunities. Your task is to analyze job descriptions
and user resumes to:

1. Optimize the user's resume to better align with the job description
2. Generate a compelling, personalized cover letter

Resume Optimization Guidelines
------------------------------
When optimizing a resume:

- Maintain complete truthfulness while strategically presenting the user's experience, skills, and 
  qualifications to highlight relevance to the target position
- Identify and incorporate key terms, skills, and qualifications from the job description
- Reorganize content to prioritize the most relevant experiences and accomplishments
- Enhance bullet points to emphasize achievements and quantifiable results that align with job requirements
- Standardize formatting and improve readability
- Suggest sections to add, expand, condense, or remove based on relevance
- Use industry-specific terminology appropriate for the role and company
- Ensure all changes remain faithful to the user's actual experience and qualifications

Cover Letter Creation Guidelines
--------------------------------
When creating a cover letter:

- Personalize the letter to the specific company, role, and job description
- Begin with a compelling introduction that shows enthusiasm for the role and company
- Highlight 3-4 of the user's most relevant experiences, skills, or achievements that directly relate
  to key requirements in the job description
- Draw clear connections between the user's background and the job requirements
- Convey the user's understanding of the company's mission, values, or industry challenges when possible
- Maintain a professional yet conversational tone appropriate for the industry and role
- Include a strong closing paragraph that expresses interest in further discussion and includes a call to action
- Keep the letter concise (250-400 words) and impactful
- Provide the cover letter in a ready-to-use format that requires minimal editing

Process
-------
1. Carefully analyze the job description to identify:
    - Required and preferred skills, experiences, and qualifications
    - Key responsibilities and expectations
    - Company values and culture indicators
    - Industry-specific terminology and keywords
2. Review the user's resume to understand their:
    - Professional background and progression
    - Core competencies and specialized skills
    - Notable achievements and impacts
    - Education, certifications, and credentials
3. Optimize the resume by:
    - Recommending specific content changes with before/after examples
    - Suggesting reorganization of information when beneficial
    - Enhancing descriptions to incorporate relevant keywords and demonstrate alignment
    - Providing a complete revised version that maintains truthfulness
4. Create a tailored cover letter that:
    - Directly addresses the hiring manager or appropriate recipient when known
    - Strategically highlights the most relevant aspects of the user's background
    - Demonstrates clear understanding of the position and enthusiasm for the opportunity
    - Presents a compelling case for why the user is an excellent candidate
    - Encourages further consideration and action from the recipient

Important Considerations
------------------------
- Maintain absolute truthfulness – never fabricate experience, skills, or qualifications
- Balance keyword optimization with natural language to avoid appearing artificially optimized
- Consider both human readers and applicant tracking systems in your optimizations
- Adapt tone and style to match the industry, role level, and company culture
- Focus on quality over quantity in both the resume and cover letter
- When information is limited, ask clarifying questions rather than making assumptions

Always adapt your approach to the specific context of each job application while maintaining the highest standards of professionalism and truthfulness.
"""

## Create a résumé prompt function

We’ll also need a résumé prompt. It’s the user prompt -— the actual input or question that you, as a user, send to the LLM. For this app, it will be the request to customize the résumé to better match the job description.

Let’s create a function that takes the base resume and the job description and outputs a user prompt asking the LLM to fine-tune the résumé.

In [ ]:
def create_resume_prompt(base_resume, job_description):
    return f"""
        I need you to optimize a resume to better match a job description. The goal is to present the candidate's experience in a way that shows they're an excellent fit for the position, while remaining truthful and professional.

            Here's the job description:
            ```
            {job_description}
            ```

            Here's the original resume in Markdown format:
            ```
            {base_resume}
            ```

            Please analyze the job description, identify key requirements and skills, and modify the resume to highlight relevant experience and accomplishments that match these requirements.

            Rules:
            1. Keep the same basic structure and sections of the original resume
            2. Don't fabricate experience or qualifications that don't exist in the original
            3. Prioritize and emphasize relevant skills and experiences
            4. Use strong action verbs and quantify achievements where possible
            5. Ensure the resume is clear, concise, and professional
            6. Maintain the Markdown formatting in your response
            7. Return ONLY the optimized resume, without any explanations

            Return the optimized resume in Markdown format.
    """

### Create a function to generate a customized resume

This function will take the following inputs:

1. Base résumé
2. Job description
3. System prompt
4. LLM model (defaults to `"gpt-4o-mini"`)
5. Temperature (defaults to `1.0`)

It outputs a new résumé that is based on the base résumé, but fine-tuned to match the job description, in Markdown format. 

In [ ]:
def generate_customized_resume(
    base_resume,
    job_description,
    system_prompt,
    model="gpt-4o-mini",
    temperature=1.0
):
    return create_completion(
        system_prompt,
        create_resume_prompt(base_resume, job_description),
        model,
        temperature
    )

Now that we have the `generate_customized_resume()` function, let’s run it and store the result in a variable named `modified_resume`. Be patient when you run it; the process will typically take 12 - 25 seconds, but can sometimes take longer.

In [ ]:
print("Generating your customized resume...")
modified_resume = generate_customized_resume(BASE_RESUME, JOB_DESCRIPTION, SYSTEM_PROMPT)
print("Done!")

## Display the fine-tuned résumé and save it as a PDF

First, let’s get a look at that résumé:

In [ ]:
print(modified_resume)

Now let’s save it as a PDF:

In [ ]:
def save_as_pdf(text, filename):
    text_lines = text.splitlines()
    if text_lines[0][0:3] == "```":
        del text_lines[0]
    
    text_for_pdf = "\n".join(text_lines)

    pdf = MarkdownPdf()
    pdf.add_section(
        Section(f"{text_for_pdf}")
    )
    try:
        pdf.save(f"{filename}.pdf")
    except Exception as pdf_exception:
        print(f"Error saving PDF: {pdf_exception}")

In [ ]:
save_as_pdf(modified_resume, "modified resume")

## How about a cover letter?

We can argue _ad infinitum_ about the value of cover letters, but let’s just concern ourselves with the fact that many job application processes _require_ a cover letter. Let’s apply the techniques we used to generate a fine-tuned résumé to generate a cover letter to go along with it. 

### Create a résumé prompt function

First, we’ll make a function that creates one using the following inputs:

- `resume`: You could use the base résumé, but it would be far better if you used the modified resume, since it’s what you’re going to provide with the cover letter.
- `job_description`: The same job description you used for the résumé.
- `company_name`: The name of the company to whom you’re sending the cover letter. The LLM will probably mention the company name in the cover letter it generates.
- `recipient_name`: Optional — if you know the recipient’s name, include it here.
- `additional_context`: If there’s anything that the LLM should know about you, your experience, or your skills that you think should be mentioned in the cover letter, include it here.

It should output a prompt instructing the LLM to generate an appropriate cover letter.

In [ ]:
def create_cover_letter_prompt(
    resume, 
    job_description, 
    company_name, 
    recipient_name="Unknown", 
    additional_context=""
):
    return f"""
        Context
        -------
        You are tasked with generating a highly personalized cover letter that connects a candidate's qualifications
        to a specific job opportunity. Use the provided resume and job description to create a persuasive, professional
        letter that positions the candidate as an ideal match.

        Input Information
        -----------------
        Job Description:
        {job_description}
        
        Resume:
        {resume}
        
        - Company Name: {company_name}
        - Recipient Name: {recipient_name}
        - Additional Context: 
          {additional_context}
        
        Instructions
        ------------
        Create a tailored cover letter that:

        - Opens with a compelling introduction that mentions the specific position and briefly explains the candidate's interest
        - Identifies 2-3 key job requirements and directly connects them to specific experiences or skills from the resume
        - Demonstrates understanding of the company's mission, values, or recent achievements (if provided)
        - Explains why the candidate would be an excellent cultural fit based on their background and the company's environment
        - Closes with enthusiasm for the opportunity and a clear call to action

        Requirements
        ------------
        - Maintain a professional yet conversational tone appropriate for the industry and position level
        - Be concise (350-450 words) and focused only on the most relevant qualifications
        - Use concrete examples and metrics from the resume when applicable
        - Avoid generic template language and empty flattery
        - Structure with proper business letter formatting including date, addresses, greeting, and signature
        - Match the candidate's experience level in language and tone (entry-level, mid-career, or executive)
        - Emphasize transferable skills when direct experience is limited
        - Include only truthful information present in the resume

        Output Format
        -------------
        Return a properly formatted business letter in Markdown that includes:

        - Current date
        - Recipient's information (if provided)
        - Professional greeting
        - 3-4 well-structured paragraphs
        - Professional closing
        - Candidate's name as signature

        The letter should read as if written by the candidate themselves, reflecting their genuine interest and qualifications.
    """

### Create a function to generate a cover letter

Now that we have a cover letter prompt function, let’s create a function to actually generate the cover letter. Just as `generate_customized_resume()` is a wrapper around the `create_completion()` function to generate a résumé, this function, `generate_cover_letter()`, is the cover letter version.

It takes the following inputs:

1. Résumé (preferably the fine-tuned one)
2. Job description
3. Company name
4. Recipient name (defaults to `"Unknown"`)
5. Additional context (defaults to `""`)
6. System prompt (defaults to `SYSTEM_PROMPT`)
7. LLM model (defaults to `"gpt-4o-mini"`)
8. Temperature (defaults to `1.0`)

It outputs a cover letter.

In [ ]:
def generate_cover_letter(
    resume,
    job_description,
    company_name,
    recipient_name="Unknown", 
    additional_context="",
    system_prompt=SYSTEM_PROMPT,
    model="gpt-4o-mini",
    temperature=1.0
):
    return create_completion(
        system_prompt,
        create_cover_letter_prompt(resume, job_description, company_name, recipient_name, additional_context),
        model,
        temperature
    )

In [ ]:
cover_letter = generate_cover_letter(
    modified_resume, 
    JOB_DESCRIPTION, 
    "RunPod", 
    None, 
    "I have 10+ years experience in Python programming.")

Let’s get a look at the cover letter:

In [ ]:
print(cover_letter)

Let’s save that cover letter as a PDF:

In [ ]:
save_as_pdf(cover_letter, "cover letter")

## Scaling up

<img src="./images/now_lets_make_it_scale.png" width="800" />

Right now, we’ve got code that generates a single résumé and a single cover letter. What we _really_ need is a way to provide code with a lot of job descriptions and then have it generate résumés and cover letters for them in a single action. Let’s make that happen!

### Reading a folder of job descriptions

The function below, given the path for a folder (a.k.a. directory) containing Markdown files (with the `.md` filename extension), returns a dictionary where:

- The **keys** are the names of the `.md` files in that folder, and
- the **values** are the corresponding contents of those files.

In [ ]:
def get_job_description_files_and_contents(directory_path):

    markdown_contents = {}
    
    # Check if the directory exists
    if not os.path.isdir(directory_path):
        print(f"Error: Directory '{directory_path}' does not exist.")
        return markdown_contents
    
    # Iterate through all files in the directory
    for filename in os.listdir(directory_path):
        # Check if the file has a .md extension
        if filename.endswith('.md'):
            file_path = os.path.join(directory_path, filename)
            
            # Read the file contents
            try:
                with open(file_path, 'r', encoding='utf-8') as file:
                    markdown_contents[filename] = file.read()
            except Exception as e:
                print(f"Error reading {filename}: {str(e)}")
    
    return markdown_contents

Let’s try it out on the `job descriptions` folder, which I’ve seeded with five actual job descriptions from LinkedIn (I copied them from LinkedIn and formatted them as Markdown):

In [ ]:
print(get_job_description_files_and_contents("./job descriptions"))

### Create a function to create résumés and cover letters 

This function will use the `get_job_description_files_and_contents()` function to generate a dictionary of resumes and cover letters where:

- The **keys** are the names of the company and the position being applied for, and 
- the **values** are dictionaries, each one containing a dictionary with the following key-value pairs:
    - `"resume"`
    - `"cover_letter"` 

In [ ]:
def create_resumes_and_cover_letters(base_resume, job_description_files):
    resumes_and_cover_letters = {}

    job_description_files = get_job_description_files_and_contents(job_description_files)

    for job_description_file in job_description_files:
        company_name, position = job_description_file.split("--")
        company_name = company_name.strip()
        position = position.replace(".md", "").strip()
        job_description = job_description_files[job_description_file]

        print(f"Generating customized resume and cover letter for {company_name} -- {position}...")

        resume = generate_customized_resume(base_resume, job_description, SYSTEM_PROMPT)
        cover_letter = generate_cover_letter(modified_resume, job_description, position, "Hiring Manager")
        resume_and_cover_letter = {"resume": resume, "cover_letter": cover_letter}
        resumes_and_cover_letters[f"{company_name} -- {position}"] = resume_and_cover_letter

    print("Done!")
    return resumes_and_cover_letters

Now that we have the `create_resumes_and_cover_letters()` function, let’s use it!

In [ ]:
resumes_and_cover_letters = create_resumes_and_cover_letters(BASE_RESUME, "./job descriptions")

Once `create_resumes_and_cover_letters()` has done its thing, let’s look at what it generated:

In [ ]:
print(resumes_and_cover_letters)

That’s a lot of stuff. Let’s look at just the keys:

In [ ]:
print(resumes_and_cover_letters.keys())

Let’s get a look at the résumé generated for that Senior Developer Advocate position at Datadog:

In [ ]:
print(resumes_and_cover_letters['Datadog -- Senior Developer Advocate - Technical Storytelling']['resume'])

Now let’s look at the matching cover letter:

In [ ]:
print(resumes_and_cover_letters['Datadog -- Senior Developer Advocate - Technical Storytelling']['cover_letter'])

And finally, let’s turn them all into PDFs:

In [ ]:
for item in resumes_and_cover_letters:
    save_as_pdf(resumes_and_cover_letters[item]["resume"], f"./output/{item} resume")
    save_as_pdf(resumes_and_cover_letters[item]["cover_letter"], f"./output/{item} cover letter")